In [1]:
# examples with classes

class Calculator:
    def __init__(self, name):
        self.name = name  # Store data in this instance
    
    def add(self, a, b):
        print(f"{self.name} is adding {a} + {b}")  # Access self.name
        return a + b

# Usage:
calc = Calculator("MyCalc")
summy = calc.add(2, 3)  # Python automatically passes calc as self
print(f"The computed sum is {summy}\n")

class Duck:
    def __init__(self, name):
        self.name = name  # Store the duck's name
    
    def introduce(self):
        print(f"Quack! My name is {self.name}")
    
    def quack(self):
        print("Quack!")

# Usage examples:
donald = Duck("Donald")
donald.introduce()
donald.quack()

MyCalc is adding 2 + 3
The computed sum is 5

Quack! My name is Donald
Quack!


In [2]:
# gradient descent basic example.
# Not clear how "non-analytical" functions can get until torch stops working...
# copilot claims torch ONLY uses analytical stuff, even for eigenvalues...??
# general goot practice: prefer torch functions

import numpy as np
import torch

#possible cost functions
def cost_function_1(x):
    return x[0]**2 + x[1]**4 + (x[0] + x[1])**6 + x[2]**2 + x[3]**2

def cost_function_2(x): # with eigenvalues involved
    # Reshape x into a matrix
    A = x.view(2, 2)  # If x has 4 elements
    eigenvals = torch.linalg.eigvals(A)
    eigenvals_real = torch.real(eigenvals)
    return torch.sum(eigenvals_real**2)

def cost_function_3(x):
    return max(x)

d = 2
cost_function = cost_function_1
# define variable to be optimized
x = torch.tensor([1.0, -2.0, 5., 3.14], dtype=torch.double, requires_grad=True)
# define optimizer
learning_rate = 0.01
n_steps = 10000
optimizer = torch.optim.Adam([x], lr=learning_rate)
for step in range(n_steps):
    optimizer.zero_grad() # clear previous gradient
    loss = cost_function(x)
    loss.backward() # compute gradient
    optimizer.step() # update x
    if step % 1000 == 0:
        print(f"x = {x.tolist()}, cost function = {loss}")


x = [1.009999999975, -1.9900000000026317, 4.99000000001, 3.1300000000159236], cost function = 52.8596
x = [0.005897630008671893, -0.2904937204018134, 0.13457109604084116, 0.001668087453219903], cost function = 0.026028579594744642
x = [0.00037449115354536817, -0.16550934894657934, 3.226168500859898e-05, 3.5483309608153823e-12], cost function = 0.0007721943108706685
x = [5.4498576897916806e-05, -0.11256823123990456, 2.5927969375248464e-12, 7.114101782273717e-33], cost function = 0.00016282688486587434
x = [1.097225479559197e-05, -0.08172503814583272, -3.4529096537761817e-28, 1.122277847165345e-55], cost function = 4.496092716242442e-05
x = [2.5850164314183053e-06, -0.06122064525186035, 4.1293257701407547e-51, 2.185807862584084e-78], cost function = 1.4115670458544878e-05
x = [6.622443599310257e-07, -0.04663116760110854, 5.564185457147956e-74, 1.3905268387082256e-101], cost function = 4.743645877179094e-06
x = [1.777910982729993e-07, -0.03585051583371858, -6.917299300598905e-97, -5.50281

In [ ]:
import sys
import os
import numpy as np
import torch

# For Jupyter notebooks, use "relative path"
sys.path.insert(0, os.path.join('..', 'src'))

from kaustav_conj.utils import h, H, nK, block_spec, M_to_A
from kaustav_conj.core import build_cost_function


n = [0.2, 0.4]
lamb = 1
# optimal rotation is e.g. a hadamard, yielding maximally mixed block spectrum [0.3, 0.3]
# so optimal M should be s.t. M_to_A(M) = log(hadamard). here is one possibility:
# M_optimal = torch.tensor([[ 0.4601,  0.0000], [-1.1107,  2.6815]], dtype=torch.double, requires_grad=True)

cost_function = build_cost_function(n, lamb)

# initialize variable to be optimized: here are some choices
# M = torch.tensor([[0., 0.1], [0., 0.]], dtype=torch.double, requires_grad=True) # start with U0 close to identity
# M = M_optimal
M = torch.rand(2, 2) - 0.5 * torch.ones(2,2)
M.requires_grad_(True)

# define optimizer
learning_rate = 0.01
n_steps = 3001
optimizer = torch.optim.Adam([M], lr=learning_rate)
for step in range(n_steps):
    optimizer.zero_grad() # clear previous gradient
    loss = cost_function(M)
    loss.backward() # compute gradient
    optimizer.step() # update x
    if step % 1000 == 0:
        print(f"Step # {step}")
#        print(f"M = {M.tolist()}, cost function = {loss}")

# print optimized unitary and block spec
U_best = torch.matrix_exp(M_to_A(M))
D = torch.diag(torch.tensor(n, dtype=torch.cdouble))
b_best = torch.real(block_spec(U_best @ D @ U_best.adjoint(), lamb))
H = torch.tensor([[1., 1.], [1., -1.]]/np.sqrt(2), dtype=torch.double)
print(f"\n The optimized unitary is \n {U_best.data} \n ")
print(f"Absolute values of its entries: \n {torch.abs(U_best).data} \n ")
print(f"Compare to Hadamard: \n {H.data} \n ")
print(f"The optimized block spectrum is \n {b_best.data}")



Step # 0
Step # 1000
Step # 2000
Step # 3000

 The optimized unitary is 
 tensor([[ 0.6920+0.1455j,  0.5108-0.4890j],
        [-0.3043-0.6383j,  0.6986+0.1092j]], dtype=torch.complex128) 
 
Absolute values of its entries: 
 tensor([[0.7071, 0.7071],
        [0.7071, 0.7071]], dtype=torch.float64) 
 
Compare to Hadamard: 
 tensor([[ 0.7071,  0.7071],
        [ 0.7071, -0.7071]], dtype=torch.float64) 
 
The optimized block spectrum is 
 tensor([0.3000, 0.3000], dtype=torch.float64)


In [11]:
d = 8
n = np.random.rand(d)
lamb = 5
learning_rate = 0.01
n_steps = 1001

# get conjectured bbest
n_conj = nK(n, lamb)

# build cost function
cost_function = build_cost_function(n, lamb)

# initialize variable to be optimized
M = torch.rand(d, d) - 0.5 * torch.ones(d,d)
M.requires_grad_(True)

# define optimizer
optimizer = torch.optim.Adam([M], lr=learning_rate)
for step in range(n_steps):
    optimizer.zero_grad() # clear previous gradient
    loss = cost_function(M)
    loss.backward() # compute gradient
    optimizer.step() # update x
    if step % 100 == 0:
        print(f"Step # {step}")
#        print(f"M = {M.tolist()}, cost function = {loss}")

# print optimized (unitary and) block spec
U_best = torch.matrix_exp(M_to_A(M))
D = torch.diag(torch.tensor(n, dtype=torch.cdouble))
b_best = torch.real(block_spec(U_best @ D @ U_best.adjoint(), lamb))
b_best_sorted = torch.cat([
    torch.sort(b_best[:lamb], descending=True).values,
    torch.sort(b_best[lamb:], descending=True).values
]).detach().numpy()
# print(f"\n The optimized unitary is \n {U_best.data} \n ")
print(f"The numerically optimized block spectrum is \n {b_best_sorted}")
print(f"Compare with the conjectured value: \n {n_conj.round(5)}")
norm_diff = np.linalg.norm(b_best_sorted - n_conj)
print(f"Norm of difference: \n {norm_diff}")

Step # 0
Step # 100
Step # 200
Step # 300
Step # 400
Step # 500
Step # 600
Step # 700
Step # 800
Step # 900
Step # 1000
The numerically optimized block spectrum is 
 [0.60126569 0.52562717 0.49598848 0.47878389 0.3906772  0.52562717
 0.49598848 0.47878389]
Compare with the conjectured value: 
 [0.60127 0.52563 0.49599 0.47878 0.39068 0.52563 0.49599 0.47878]
Norm of difference: 
 1.877551628332819e-09


In [1]:
import sys
import os
import numpy as np
import torch

# For Jupyter notebooks, use "relative path"
sys.path.insert(0, os.path.join('..', 'src'))

from kaustav_conj.utils import h, H, nK, block_spec, M_to_A
from kaustav_conj.core import build_cost_function, get_b_best

n = [0.2, 0.6]
lamb = 1
b_best_conj = nK(n, lamb)
U_best, b_best_num, H_best, conjecture_holds = get_b_best(n, lamb, N_init=4, N_steps=300,learning_rate=0.01)
np.allclose(b_best_conj, b_best_num, rtol=1e-6) and conjecture_holds



Starting get_b_best optimization
Parameters recap:
  n = [0.2, 0.6]
  lambda = 1
  rand_range = 1.0
  N_init = 4
  N_steps = 300
  learning_rate = 0.01
  eps = 1e-12


Starting gradient descent run 1/4
Gradient descent, step # 0
Gradient descent, step # 100
Gradient descent, step # 200

Results of gradient descent run 1/4
Numerical b_best: 
 [0.40000002 0.39999998]
Conjectured b_best: 
 [0.4 0.4]
Norm of difference: 
 2.529284621679259e-08
Conjectured H_best - numerical H_best (should be > 0): 
 1.7763568394002505e-15
Conjectured majorization: 
 True

Starting gradient descent run 2/4
Gradient descent, step # 0
Gradient descent, step # 100
Gradient descent, step # 200

Results of gradient descent run 2/4
Numerical b_best: 
 [0.39999996 0.40000004]
Conjectured b_best: 
 [0.4 0.4]
Norm of difference: 
 5.6811281232156754e-08
Conjectured H_best - numerical H_best (should be > 0): 
 6.661338147750939e-15
Conjectured majorization: 
 True

Starting gradient descent run 3/4
Gradient descent,

True

In [5]:
d = 14
lamb = 10
n = np.random.rand(d)
N_init = 1
N_steps = 2000
learning_rate = 0.005
eps = 1e-13
b_best_conj = nK(n, lamb)
U_best, b_best_num, H_best, conjecture_holds = get_b_best(n, lamb, N_init=N_init, N_steps=N_steps,learning_rate=learning_rate, eps=eps)


Starting get_b_best optimization
Parameters recap:
  n = [0.2229409  0.05596309 0.1386509  0.80645666 0.91983109 0.81623408
 0.30484322 0.99087029 0.57684608 0.68341568 0.45366076 0.33017196
 0.80776949 0.85441828]
  lambda = 10
  rand_range = 1.0
  N_init = 1
  N_steps = 2000
  learning_rate = 0.005
  eps = 1e-13


Starting gradient descent run 1/1
Gradient descent, step # 0
Gradient descent, step # 100
Gradient descent, step # 200
Gradient descent, step # 300
Gradient descent, step # 400
Gradient descent, step # 500
Gradient descent, step # 600
Gradient descent, step # 700
Gradient descent, step # 800
Gradient descent, step # 900
Gradient descent, step # 1000
Gradient descent, step # 1100
Gradient descent, step # 1200
Gradient descent, step # 1300
Gradient descent, step # 1400
Gradient descent, step # 1500
Gradient descent, step # 1600
Gradient descent, step # 1700
Gradient descent, step # 1800
Gradient descent, step # 1900

Results of gradient descent run 1/1
Numerical b_best: 
 [0

In [2]:
# now for a rough way how one can test conjecture across various d and lamb values. should parallelize or use cluster instead!
d_max = 5
conjecture_holds = True
# TODO: make N_n, N_init, (N_steps, learning_rate) d-dependent, i.e. upgrade them to lists of length d_max - 1
N_n = 10
N_init = 4
N_steps = 1000
learning_rate = 0.01
eps = 1e-13
for d in range(1, d_max + 1):
    for lamb in range(int(np.ceil(d/2)), d):
        print(F"\n{'='*40}\n{'='*40}\nCASE d = {d}, lambda = {lamb}\n{'='*40}\n{'='*40}\n")
        for _ in range(N_n):
            n = np.random.rand(d)
            b_best_conj = nK(n, lamb)
            U_best, b_best_num, H_best, conjecture_holds = get_b_best(n, lamb, N_init=N_init, N_steps=N_steps,learning_rate=learning_rate, eps=eps)
            if conjecture_holds == False:
                break
        if conjecture_holds == False:
            break
    if conjecture_holds == False:
        break
            


CASE d = 2, lambda = 1


Starting get_b_best optimization
Parameters recap:
  n = [0.24289581 0.33596876]
  lambda = 1
  rand_range = 1.0
  N_init = 4
  N_steps = 1000
  learning_rate = 0.01
  eps = 1e-13


Starting gradient descent run 1/4
Gradient descent, step # 0
Gradient descent, step # 100
Gradient descent, step # 200
Gradient descent, step # 300
Gradient descent, step # 400
Gradient descent, step # 500
Gradient descent, step # 600
Gradient descent, step # 700
Gradient descent, step # 800
Gradient descent, step # 900

Results of gradient descent run 1/4
Numerical b_best: 
 [0.28943228 0.28943228]
Conjectured b_best: 
 [0.28943228 0.28943228]
Norm of difference: 
 1.5803773493345781e-10
Conjectured H_best - numerical H_best (should be > 0): 
 -2.220446049250313e-16
Conjectured majorization: 
 True

Starting gradient descent run 2/4
Gradient descent, step # 0
Gradient descent, step # 100
Gradient descent, step # 200
Gradient descent, step # 300
Gradient descent, step # 400
Gradien

In [3]:
conjecture_holds

True